In [1]:
import sys

# IT'S 2025 WHY HASN'T SOMEONE FIXED THIS
if ".." not in sys.path:
    sys.path.insert(0, "..")
    
import pandas as pd
import numpy as np
import ast

import tasks.run_helper

In [2]:
df = pd.read_csv(
    "../data/100_annotators.csv",
    converters={"annot_personality_characteristics": ast.literal_eval},
)
df["toxicity"] = df.annotation.apply(lambda x: x[-1]).astype(int)
df["annot_politics"] = df.annot_personality_characteristics.apply(
    lambda x: x[0]
)
df.annot_age = pd.cut(df.annot_age, bins=4)
df.message_id = df.message_id.astype(str)
df["comment_key"] = df.conv_id + df.message_id

df = df.loc[
    :,
    [
        "conv_id",
        "message_id",
        "comment_key",
        "message",
        "toxicity",
        "annot_age",
        "annot_sex",
        "annot_sexual_orientation",
        "annot_demographic_group",
        "annot_current_employment",
        "annot_education_level",
        "annot_politics",
    ],
]
df = df.groupby(["conv_id", "message_id", "comment_key", "message"]).apply(
    lambda x: pd.Series(
        {
            col: x[col].tolist()
            for col in df.columns
            if col not in ["conv_id", "message_id", "comment_key", "message"]
        }
    ),
    include_groups=False,
)
df.head(1)

,,,,toxicity,annot_age,annot_sex,annot_sexual_orientation,annot_demographic_group,annot_current_employment,annot_education_level,annot_politics
conv_id,message_id,comment_key,message,,,,,,,,
250cc245-86c4-4e97-a8ce-31d5d6796f71,-1428289379724132680,250cc245-86c4-4e97-a8ce-31d5d6796f71-1428289379724132680,"well thats a no brainer hillary clinton gave huma abdein a security clearance when she has ties to a known terrorist group the muslim brotherhood, and her mother runs an anti american news paper in the middle east, debbie washed up crook shultz got the awan famaily security clearances and they were recent immigrants, had absolutely no IT experience and possible ties to terrorist groups in pakistan. its pretty clear our liberal ran government is a complete and total failure when it comes to national security. 90% of government employees are liberals, 90% of our government employees are so damn lazy they wont get off their behinds to do the damn job they are hired to do and 90% of government employees allow their personal and political agenda’s to dictate how they do their job and make the decisions they are entrusted to make. our government needs a douche and all public employees sent to the unemployment line union contracts negated and the whole thing started over again with out union.\n","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[(69.75, 88.0], (33.25, 51.5], (69.75, 88.0], ...","[female, non-binary, female, male, male, male,...","[homosexual, straight, straight, straight, bis...","[black, white, asian, other, asian, other, asi...","[blue-collar, white-collar, unemployed, white-...","[university, none, university, none, universit...","[right-wing conservative, right-wing conservat..."


In [6]:
for discussion_id in df.reset_index().conv_id.unique():
    print(10 * "=")
    print(discussion_id)
    discussion_df = df
    discussion_df = discussion_df.reset_index()
    discussion_df = discussion_df[discussion_df.conv_id == discussion_id]
    print(
        tasks.run_helper.run_aposteriori(
            discussion_df,
            feature_col="annot_age",
            value_col="toxicity",
            comment_key_col="comment_key",
        )
    )
    print(
        tasks.run_helper.run_aposteriori(
            discussion_df,
            feature_col="annot_sex",
            value_col="toxicity",
            comment_key_col="comment_key",
        )
    )
    print(
        tasks.run_helper.run_aposteriori(
            discussion_df,
            feature_col="annot_demographic_group",
            value_col="toxicity",
            comment_key_col="comment_key",
        )
    )
    print(
        tasks.run_helper.run_aposteriori(
            discussion_df,
            feature_col="annot_sexual_orientation",
            value_col="toxicity",
            comment_key_col="comment_key",
        )
    )
    print(
        tasks.run_helper.run_aposteriori(
            discussion_df,
            feature_col="annot_current_employment",
            value_col="toxicity",
            comment_key_col="comment_key",
        )
    )
    print(
        tasks.run_helper.run_aposteriori(
            discussion_df,
            feature_col="annot_education_level",
            value_col="toxicity",
            comment_key_col="comment_key",
        )
    )
    print(
        tasks.run_helper.run_aposteriori(
            discussion_df,
            feature_col="annot_education_level",
            value_col="toxicity",
            comment_key_col="comment_key",
        )
    )

250cc245-86c4-4e97-a8ce-31d5d6796f71
(14.927, 33.25]    0.10809
(33.25, 51.5]      1.00000
(51.5, 69.75]      1.00000
(69.75, 88.0]      1.00000
dtype: float64
female        1.000000
male          1.000000
non-binary    0.336996
dtype: float64
asian    1.0
black    1.0
other    1.0
white    1.0
dtype: float64
bisexual      1.0
homosexual    1.0
other         1.0
straight      1.0
dtype: float64
blue-collar     0.873061
unemployed      1.000000
white-collar    1.000000
dtype: float64
high-school    1.0
none           1.0
university     1.0
dtype: float64
high-school    1.0
none           1.0
university     1.0
dtype: float64
2feee653-15e9-4c38-b7a2-96fe26f50c9f
(14.927, 33.25]    1.000000
(33.25, 51.5]      0.494639
(51.5, 69.75]      1.000000
(69.75, 88.0]      1.000000
dtype: float64
female        1.0
male          1.0
non-binary    1.0
dtype: float64
asian    1.000000
black    0.928741
other    1.000000
white    1.000000
dtype: float64
bisexual      1.0
homosexual    1.0
other       